야후 API에서 제공하는 기업 설명을 따로 supabase의 저장소에 추가

In [48]:
#라이브러리
##기본 작업
import os
import pandas as pd
import numpy as np
import time
import json
from datetime import datetime, timedelta
##뉴스스크랩
import re
import requests
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from webdriver_manager.chrome import ChromeDriverManager
from bs4 import BeautifulSoup
from dateutil.parser import parse
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.by import By
from selenium.common.exceptions import TimeoutException
## API 구동
import http.client
import uuid
from supabase import create_client, Client

# supabase 연결

In [7]:
import os
import pandas as pd
from dotenv import load_dotenv
from supabase import create_client, Client
# 상위 폴더의 .env 파일 경로를 지정하여 로드
load_dotenv(dotenv_path='../.env')
# 환경 변수 사용
supabase_url = os.environ.get("SUPABASE_URL")
supabase_key = os.environ.get("SUPABASE_KEY")
supabase: Client = create_client(supabase_url, supabase_key)

# 야후 API

In [5]:
import yfinance as yf

# 한국 주식 목록
Name_KR=['Samsung', 'SK Hynix', 'Samsung Biologics', 'LG Energy Solution', 'Hyundai', 'KB Financial Group', 'Celltrion', 'Doosan Enerbility', 'Kia', 'Hanwha Aerospace', 'Naver', 'HD Hyundai Heavy Industries', 'Shinhan Financial Group', 'Hyundai Mobis', 'Kakao', 'HMM', 'Hana Financial Group', 'POSCO', 'Korea Electric Power', 'Samsung Life Insurance', 'Hanwha Ocean (DSME)', 'HD Korea Shipbuilding & Offshore Engineering', 'SK Square', 'Meritz Financial Group', 'Alteogen', 'Hyundai Rotem Company', 'LG Chem', 'SK Innovation', 'Woori Financial Group', 'Krafton', 'HD Hyundai Electric', 'Industrial Bank of Korea (IBK)', 'Korea Zinc', 'Samsung SDI', 'KakaoBank', 'KT Corporation', 'KT&G (Korea Tobacco)', 'Samsung Heavy Industries', 'LG Electronics', 'LG Corp', 'SK Telecom', 'HYBE', 'SK Group', 'LIG Nex1', 'POSCO Chemical', 'Kakao Pay', 'Samsung Electro-Mechanics', 'Hyundai Glovis', 'Ecopro BM', 'Hanwha Systems', 'Samyang Foods', 'HANMI Semiconductor', 'Doosan', 'Korean Air Lines', 'Posco International', 'HD Hyundai Marine Solution', 'Korea Aerospace Industries', 'Amorepacific', 'Yuhan', 'S-OIL', 'Coway', 'Ecopro', 'Hanwha Solutions', 'Doosan Bobcat', 'Hankook Tire', 'Netmarble', 'LG Household & Health Care', 'LG Display', 'LS Corp.', 'ORION', 'Korea Gas', 'CJ Group', 'NCsoft', 'Kangwon Land', 'LEENO Industrial', 'F&F Co', 'SM Entertainment', 'Kumho Petrochemical', 'Hanmi Pharmaceutical', 'LOTTE Chemical', 'SHIFT UP', 'Pearl Abyss', 'JYP Entertainment', 'KCC Corp', 'Nongshim', 'Fila', 'BGF Retail', 'Pan Ocean', 'LOTTE Corporation', 'Asiana Airlines', 'Hansol Chemical', 'Shinsegae', 'YG Entertainment', 'Doosan Fuel Cell', 'Kakao Games', 'HiteJinro', 'Kumho Tire', 'Ottogi', 'SD BioSensor', 'DoubleUGames']
Ticker_KR=['005930.KS', '000660.KS', '207940.KS', '373220.KS', 'HYMTF', 'KB', '068270.KS', '034020.KS', '000270.KS', '012450.KS', '035420.KS', '329180.KS', 'SHG', '012330.KS', '035720.KS', '011200.KS', '086790.KS', 'PKX', 'KEP', '032830.KS', '042660.KS', '009540.KS', '402340.KS', '138040.KS', '196170.KQ', '064350.KS', '051910.KS', '096775.KS', 'WF', '259960.KS', '267260.KS', '024110.KS', '010130.KS', '006405.KS', '323410.KS', 'KT', '033780.KS', '010140.KS', '066570.KS', '003550.KS', 'SKM', '352820.KS', '034730.KS', '079550.KS', '003670.KS', '377300.KS', '009155.KS', '086280.KS', '247540.KQ', '272210.KS', '003230.KS', '042700.KS', '000150.KS', '003495.KS', '047050.KS', '443060.KS', '047810.KS', '090430.KS', '000100.KS', '010950.KS', '021240.KS', '086520.KQ', '009830.KS', '241560.KS', '161390.KS', '251270.KS', '051905.KS', 'LPL', '006260.KS', '271560.KS', '036460.KS', '001040.KS', '036570.KS', '035250.KS', '058470.KQ', '383220.KS', '041510.KQ', '011780.KS', '008930.KS', '011170.KS', '462870.KS', '263750.KQ', '035900.KQ', '002380.KS', '004370.KS', '081660.KS', '282330.KS', '028670.KS', '004990.KS', '020560.KS', '014680.KS', '004170.KS', '122870.KQ', '336260.KS', '293490.KQ', '000080.KS', '073240.KS', '007310.KS', '137310.KS', '192080.KS']
# 미국 주식 목록
Name_US=['NVIDIA', 'Microsoft', 'Apple', 'Amazon', 'Alphabet (Google)', 'Meta Platforms (Facebook)', 'Broadcom', 'Berkshire Hathaway', 'Tesla', 'JPMorgan Chase', 'Walmart', 'Visa', 'Eli Lilly', 'Oracle', 'Netflix', 'Mastercard', 'Exxon Mobil', 'Costco', 'Procter & Gamble', 'Johnson & Johnson', 'Home Depot', 'Bank of America', 'AbbVie', 'Palantir', 'Coca-Cola', 'UnitedHealth', 'Philip Morris International', 'Cisco', 'T-Mobile US', 'Wells Fargo', 'IBM', 'General Electric', 'Salesforce', 'Chevron', 'Abbott Laboratories', 'Morgan Stanley', 'American Express', 'AMD', 'Walt Disney', 'Goldman Sachs', 'Intuit', 'ServiceNow', 'McDonald', 'AT&T', 'Merck', 'Texas Instruments', 'Uber', 'Intuitive Surgical', 'RTX', 'Blackstone Group', 'Caterpillar', 'Booking Holdings (Booking.com)', 'Pepsico', 'Verizon', 'QUALCOMM', 'BlackRock', 'Charles Schwab', 'Citigroup', 'Boeing', 'S&P Global', 'Thermo Fisher Scientific', 'Adobe', 'Amgen', 'Honeywell', 'Boston Scientific', 'Progressive', 'Applied Materials', 'Nextera Energy', 'Stryker Corporation', 'Danaher', 'Pfizer', 'Capital One', 'Union Pacific Corporation', 'GE Vernova', 'Deere & Company (John Deere)', 'TJX Companies', 'Gilead Sciences', 'Micron Technology', 'Palo Alto Networks', 'Comcast', 'Arista Networks', 'KKR & Co.', 'CrowdStrike', "Lowe's Companies", 'Lam Research', 'Automatic Data Processing', 'KLA', 'Analog Devices', 'Amphenol', 'ConocoPhillips', 'Vertex Pharmaceuticals', 'AppLovin', 'Strategy \n (MicroStrategy)', 'Nike', 'Lockheed Martin', 'Starbucks', 'Marsh & McLennan Companies', 'Intercontinental Exchange', 'American Tower', 'DoorDash']
Ticker_US=['NVDA', 'MSFT', 'AAPL', 'AMZN', 'GOOG', 'META', 'AVGO', 'BRK-B', 'TSLA', 'JPM', 'WMT', 'V', 'LLY', 'ORCL', 'NFLX', 'MA', 'XOM', 'COST', 'PG', 'JNJ', 'HD', 'BAC', 'ABBV', 'PLTR', 'KO', 'UNH', 'PM', 'CSCO', 'TMUS', 'WFC', 'IBM', 'GE', 'CRM', 'CVX', 'ABT', 'MS', 'AXP', 'AMD', 'DIS', 'GS', 'INTU', 'NOW', 'MCD', 'T', 'MRK', 'TXN', 'UBER', 'ISRG', 'RTX', 'BX', 'CAT', 'BKNG', 'PEP', 'VZ', 'QCOM', 'BLK', 'SCHW', 'C', 'BA', 'SPGI', 'TMO', 'ADBE', 'AMGN', 'HON', 'BSX', 'PGR', 'AMAT', 'NEE', 'SYK', 'DHR', 'PFE', 'COF', 'UNP', 'GEV', 'DE', 'TJX', 'GILD', 'MU', 'PANW', 'CMCSA', 'ANET', 'KKR', 'CRWD', 'LOW', 'LRCX', 'ADP', 'KLAC', 'ADI', 'APH', 'COP', 'VRTX', 'APP', 'MSTR', 'NKE', 'LMT', 'SBUX', 'MMC', 'ICE', 'AMT', 'DASH']


한국, 미국기업 모두 같은 테이블에 저장할거다

In [6]:
all_names = Name_KR + Name_US
all_tickers = Ticker_KR + Ticker_US

In [9]:
company_summaries = []

for name, ticker in zip(all_names, all_tickers):
    stock = yf.Ticker(ticker)
    summary = stock.info.get('longBusinessSummary', pd.NA)
    company_summaries.append({
        'Name': name,
        'Ticker': ticker,
        'Summary': summary
    })

In [42]:
results_df = pd.DataFrame(company_summaries)

In [43]:
results_df.head()

,Name,Ticker,Summary
0,Samsung,005930.KS,"Samsung Electronics Co., Ltd. engages in the c..."
1,SK Hynix,000660.KS,"SK hynix Inc., together with its subsidiaries,..."
2,Samsung Biologics,207940.KS,"Samsung Biologics Co.,Ltd., together with its ..."
3,LG Energy Solution,373220.KS,"LG Energy Solution, Ltd. provides energy solut..."
4,Hyundai,HYMTF,"Hyundai Motor Company, together with its subsi..."


In [44]:
results_df.isna().sum()

Name       0
Ticker     0
Summary    1
dtype: int64

In [45]:
results_df[results_df['Summary'].isna()]

,Name,Ticker,Summary
61,Ecopro,086520.KQ,<NA>


# 요약 한국어로 번역

CLOVA X를 사용하여 번역을 자동화

In [46]:
import requests
import pandas as pd
import json
import time

# --- 제공해주신 코드를 기반으로 한, 정상 작동하는 Executor ---
class CompletionExecutor:
    def __init__(self, host, api_key, request_id):
        self._host = host
        self._api_key = api_key
        self._request_id = request_id

    def execute(self, completion_request):
        headers = {
            'Authorization': self._api_key,
            'X-NCP-CLOVASTUDIO-REQUEST-ID': self._request_id,
            'Content-Type': 'application/json; charset=utf-8',
            'Accept': 'text/event-stream'
        }

        with requests.post(self._host + '/testapp/v3/chat-completions/HCX-DASH-002',
                           headers=headers, json=completion_request, stream=True) as r:
            
            final_content = None # 최종 content를 저장할 변수
            
            for line in r.iter_lines():
                if not line:
                    continue

                decoded_line = line.decode('utf-8')
                
                if 'data: [DONE]' in decoded_line:
                    break
                
                if decoded_line.startswith('data:'):
                    try:
                        json_str = decoded_line[len('data:'):].strip()
                        data = json.loads(json_str)
                        
                        if 'message' in data and 'content' in data['message']:
                            # content가 비어있지 않은 경우, 이 내용을 최종 결과로 덮어씁니다.
                            if data['message']['content']:
                                final_content = data['message']['content']
                    except (json.JSONDecodeError, KeyError):
                        continue
                        
        return final_content

# --- 번역 작업을 수행하는 새로운 함수 ---
def translate_summary_corrected(summary_text):
    """
    영문 요약본을 받아 번역을 요청하고, 최종 번역 결과를 반환합니다.
    """
    if pd.isna(summary_text) or not summary_text.strip():
        return pd.NA

    # LLM API 호출을 위한 인스턴스 생성
    completion_executor = CompletionExecutor(
        host='https://clovastudio.stream.ntruss.com',
        api_key='Bearer nv-3da49e910a8b4d3b99721dab49141e35xINY', # 이것은 테스트키이므로 나중에 사업용 키를 받았을 경우 대체한다.
        request_id='YOUR_REQUEST_ID' 
    )

    # 번역을 위한 시스템 프롬프트
    preset_text = [
        {"role":"system","content":"[페르소나]\n당신은 영문 기업 정보를 한국어로 번역하는 전문 번역가입니다. 당신의 번역은 기계적이지 않으며, 원문의 의미와 뉘앙스를 깊이 이해하여 전달하는 데 초점을 맞춥니다.\n\n[주요 임무]\n주어진 영문 기업 설명을 원문의 의미를 최대한 보존하며, 한국어 사용자가 이해하기 쉬운 자연스러운 문장으로 번역합니다.\n\n[규칙 및 지침]\n\n의미 보존: 원문의 전문 용어나 사업 내용을 누락하거나 왜곡하지 않고, 의미를 그대로 전달해야 합니다.\n\n기업명 번역 형식: 기업의 이름은 반드시 '한국어 번역명(Original English Name)' 형식으로 표기해야 합니다.\n\n기업명 출처: 괄호 안에 들어갈 'Original English Name'은 제가 제공하는 설명문 본문에 포함된 기업명을 그대로 사용해야 합니다. 임의로 다른 이름을 찾거나 수정하지 마십시오.\n\n문체: 간결하고 전문적인 어조를 유지합니다.\n\n[예시]\n\n입력: Apple Inc. is a technology company that designs, develops, and sells consumer electronics, computer software, and online services.\n\n출력: 애플(Apple Inc.)은 가전제품, 컴퓨터 소프트웨어, 온라인 서비스를 설계, 개발 및 판매하는 기술 기업입니다."},
        {"role":"user","content": summary_text}
    ]

    request_data = {
        'messages': preset_text,
        'topP': 0.8,
        'topK': 0,
        'maxTokens': 512,
        'temperature': 0.5,
        'repetitionPenalty': 1.1,
        'stop': [],
        'includeAiFilters': True,
        'seed': 0
    }

    try:
        # API 실행 및 결과 받기
        result = completion_executor.execute(request_data)
        time.sleep(0.5) # API 요청 간 지연

        if result:
            return result.strip()
        else:
            return "응답 없음"

    except Exception as e:
        print(f"API 처리 중 오류 발생: {e}")
        return "오류 발생"


# --- 메인 실행 로직 ---
if __name__ == '__main__':
    print("번역을 시작합니다...")
    results_df['Summary_KR'] = results_df['Summary'].apply(translate_summary_corrected)
    print("번역 완료.")

번역을 시작합니다...
번역 완료.


In [47]:
results_df.head()

,Name,Ticker,Summary,Summary_KR
0,Samsung,005930.KS,"Samsung Electronics Co., Ltd. engages in the c...","삼성전자 주식회사(Samsung Electronics Co., Ltd.)는 소비자 ..."
1,SK Hynix,000660.KS,"SK hynix Inc., together with its subsidiaries,...","SK하이닉스㈜는 자회사와 함께 한국과 중국, 아시아 나머지 지역, 미국, 유럽에서 ..."
2,Samsung Biologics,207940.KS,"Samsung Biologics Co.,Ltd., together with its ...","삼성바이오로직스㈜는 한국과 유럽, 미국 등 전 세계에서 바이오 의약품 제조를 담당하..."
3,LG Energy Solution,373220.KS,"LG Energy Solution, Ltd. provides energy solut...","엘지 에너지 솔루션(LG Energy Solution, Ltd.)은 전 세계에 에너..."
4,Hyundai,HYMTF,"Hyundai Motor Company, together with its subsi...",현대자동차는 자회사와 함께 전 세계에서 자동차 및 부품을 제조 및 유통합니다. 이 ...


In [49]:
results_df.iloc[0]["Summary_KR"]

'삼성전자 주식회사(Samsung Electronics Co., Ltd.)는 소비자 전자제품, 정보기술 및 모바일 통신, 디바이스 솔루션 사업과 전 세계 연구 개발 센터를 운영합니다. 회사는 DX, DS, SDC 그리고 하만 등 네 가지 부문으로 나뉩니다. 이 회사는 스마트폰, 태블릿, 오디오 사운드, 시계, 스위치 및 액세서리; TV 및 음향 기기; 냉장고, 세탁기 및 건조기, 진공청소기, 조리 기구, 식기세척기, 에어컨 및 공기 청정기와 같은 가전제품; 모니터 및 메모리 저장 제품; 디스플레이 및 스마트 및 LED 간판; 기타 액세서리를 제공합니다. 또한 벤처 캐피털 투자, 클라우드 서비스, 네트워크 장치 설치, 반도체 장비 서비스, 디지털 광고 플랫폼, 마케팅, 컨설팅, 연결 서비스 제공자, 물류, 자금 조달 및 소프트웨어 디자인 활동에도 참여합니다. 반도체 및 디스플레이 패널의 통행 처리 및 판매, 네트워크 솔루션 개발 및 제조, 반도체 및 식품 생산, 전자 장치 수리 서비스 제공, 반도체 공정 결함 및 품질 관리 소프트웨어 개발 및 공급, 디지털 텔레비전, 팹, 시스템 대형 집적 회로(LSI), 커넥티드 카 시스템, 오디오 및 비디오 제품, 기업 자동화 솔루션 및 연결된 서비스에 대한 개발을 수행합니다. 1969년에 설립되었으며 대한민국 수원시에 본사를 두고 있습니다.'

# 문단 나누기

In [52]:
# API 실행 클래스
class CompletionExecutor:
    def __init__(self, host, api_key, request_id):
        self._host = host
        self._api_key = api_key
        self._request_id = request_id

    def _send_request(self, completion_request):
        headers = {
            'Content-Type': 'application/json; charset=utf-8',
            'Authorization': self._api_key,
            'X-NCP-CLOVASTUDIO-REQUEST-ID': self._request_id
        }

        conn = http.client.HTTPSConnection(self._host)
        conn.request('POST', '/serviceapp/v1/api-tools/segmentation', json.dumps(completion_request), headers)
        response = conn.getresponse()
        result = json.loads(response.read().decode(encoding='utf-8'))
        conn.close()
        return result

    def execute(self, completion_request):
        res = self._send_request(completion_request)
        if res['status']['code'] == '20000':
            # API 응답의 'topicSeg'가 이미 텍스트의 리스트이므로, 추가 처리 없이 그대로 반환합니다.
            return res['result']['topicSeg']
        else:
            print(f"API Error: {res}")
            return [] # 오류 발생 시 빈 리스트 반환


# 데이터프레임의 요약본을 문단으로 나눈다
def segment_news_summary(summary_text):
    """하나의 요약본 텍스트를 받아 문단 나누기 API를 호출하고 결과를 반환합니다."""
    if not isinstance(summary_text, str) or not summary_text.strip():
        return []

    completion_executor = CompletionExecutor(
        host='clovastudio.stream.ntruss.com',
        api_key='Bearer nv-954d9d9843374384bfe278a5169c2ee18Q6C', #테스트키이므로 후에 사업용키로 대체
        request_id='37e746486492410dab50545828f1edfb' # API 구분 ID
    )

    # API 기본 옵션
    request_data = {
        "postProcessMaxSize": 1000,
        "alpha": 0.0,
        "segCnt": -1,
        "postProcessMinSize": 300,
        "text": summary_text,
        "postProcess": False
    }
    
    # API 요청 간의 과부하를 막기 위해 약간의 지연 시간 추가
    time.sleep(0.5)
    
    return completion_executor.execute(request_data)


In [53]:
# 문단 나누기 실행
print("뉴스 요약본에 대한 문단 나누기를 시작합니다...") # 진행 확인용
# 'summary' 열의 각 텍스트에 대해 문단 나누기 함수를 적용하고, 결과를 새 열에 저장
results_df['segmented_summary'] = results_df['Summary'].apply(segment_news_summary)
results_df['segmented_summary_kr'] = results_df['Summary_KR'].apply(segment_news_summary)
print("문단 나누기 작업이 완료되었습니다.")

뉴스 요약본에 대한 문단 나누기를 시작합니다...
문단 나누기 작업이 완료되었습니다.


# 임베딩

In [54]:
# API 실행 클래스
class EmbeddingExecutor:
    def __init__(self, host, api_key):
        self._host = host
        self._api_key = api_key

    def _send_request(self, completion_request, request_id):
        headers = {
            'Content-Type': 'application/json; charset=utf-8',
            'Authorization': self._api_key,
            'X-NCP-CLOVASTUDIO-REQUEST-ID': request_id
        }
        try:
            conn = http.client.HTTPSConnection(self._host)
            conn.request('POST', '/serviceapp/v1/api-tools/embedding/clir-emb-dolphin', json.dumps(completion_request), headers)
            response = conn.getresponse()
            result = json.loads(response.read().decode(encoding='utf-8'))
            conn.close()
            return result
        except Exception as e:
            return {'status': {'code': 'N/A', 'message': str(e)}}

    def execute(self, completion_request):
        request_id = str(uuid.uuid4())
        res = self._send_request(completion_request, request_id)
        if res.get('status') and res['status'].get('code') == '20000':
            return res.get('result', {}).get('embedding', [])
        else:
            status = res.get('status', {})
            code = status.get('code', 'Unknown')
            message = status.get('message', 'No error message returned.')
            print(f"Error (Code: {code}): {message}")
            return []

# 텍스트 조각 임베딩 함수 
def embed_text_segments(segments, executor):
    if not isinstance(segments, list):
        return []
    vectors = []
    for segment in segments:
        if isinstance(segment, str) and segment.strip():
            request_data = {"text": segment.strip()}
            vector = executor.execute(request_data)
            if vector:
                vectors.append(vector)
            time.sleep(0.2)
    return vectors

# 실행
API_KEY = 'Bearer nv-954d9d9843374384bfe278a5169c2ee18Q6C' #테스트키임으로 후에 사업용키로 대체
HOST = 'clovastudio.stream.ntruss.com'

embedding_executor = EmbeddingExecutor(host=HOST, api_key=API_KEY)

In [55]:

print("임베딩을 시작합니다...")
# 문단 나누기 결과를 기반으로 임베딩 진행
results_df['Summary_embedding'] = results_df['segmented_summary'].apply(
    lambda x: embed_text_segments(sum(x, []), executor=embedding_executor)
)
results_df['Summary_KR_embedding'] = results_df['segmented_summary_kr'].apply(
    lambda x: embed_text_segments(sum(x, []), executor=embedding_executor)
)
print("임베딩 작업이 완료되었습니다.")

임베딩을 시작합니다...
Error (Code: 40003): Context length exceeded
임베딩 작업이 완료되었습니다.


# 업로드

In [ ]:
results_df.rename(columns={
    'Name': 'company_name',
    'Ticker': 'ticker',
    'Summary': 'summary',
    'Summary_KR': 'summary_kr',
    "Summary_embedding": "summary_embedding",
    "Summary_KR_embedding": "summary_kr_embedding"
}, inplace=True)
results=results_df[['company_name', 'ticker', 'summary', 'summary_kr', 'summary_embedding', 'summary_kr_embedding']].copy()

In [84]:
def average_vectors(vectors):
    if not vectors or not isinstance(vectors[0], list):
        return None
    # np.mean을 사용하여 벡터들의 평균을 계산하고 다시 리스트로 변환
    return np.mean(vectors, axis=0).tolist()
results['summary_embedding'] = results['summary_embedding'].apply(average_vectors)
results['summary_kr_embedding'] = results['summary_kr_embedding'].apply(average_vectors)

In [85]:
data_to_upload = results.to_dict('records')

In [87]:
data, count = supabase.table("company_summary").insert(data_to_upload).execute()